In [1]:
import pandas as pd

model_summaries = pd.read_csv('model_summaries.csv')
model_summaries.head(12)

,correlation.structure,has.interaction,is.missing,coefficient,estimate,standard.error
0,GEE Independent,False,False,(Intercept),-0.024537,0.070636
1,GEE Independent,False,False,age,-0.002281,0.010757
2,GEE Exchangeable,False,False,(Intercept),-0.059908,0.068872
3,GEE Exchangeable,False,False,age,0.015402,0.010418
4,Mixed Model,False,False,(Intercept),-0.126181,0.114182
5,Mixed Model,False,False,age,0.028707,0.015460
6,GEE Independent,True,False,(Intercept),0.576457,0.117464
7,GEE Independent,True,False,age,-0.048729,0.017763
8,GEE Independent,True,False,income,-0.964447,0.148575
9,GEE Independent,True,False,age:income,0.076834,0.022462


In [2]:
model_summaries['correlation.structure'] = pd.Categorical(
    model_summaries['correlation.structure'],
    pd.unique(model_summaries['correlation.structure']),
    ordered=True)
model_summaries['coefficient'] = pd.Categorical(
    model_summaries['coefficient'],
    ['(Intercept)', 'age', 'income', 'age:income'],
    ordered=True)

In [3]:
def pivot_table(data):
    pivoted = pd.pivot_table(
        data,
        values=['estimate','standard.error'],
        index=['coefficient'],
        columns=['correlation.structure'],
        aggfunc='mean')
    pivoted = pivoted.swaplevel(0, 1, 1).sort_index(1)    
    pivoted.index.rename('Coefficient', inplace=True)
    pivoted.columns.rename(['Correlation Structure', None], inplace=True)
    pivoted = pivoted.rename(
        {'estimate': 'Estimate', 'standard.error': 'Standard Error'}, axis=1)
    return pivoted.rename({
        '(Intercept)': '(Intercept), $\\beta_0$', 'age': 'age, $\\beta_1$',
        'income': 'income, $\\beta_2$', 'age:income': 'age:income, $\\beta_3$'}, axis=0)

In [4]:
model_interaction_full_data_fit = pivot_table(
    model_summaries[model_summaries['has.interaction'] & ~model_summaries['is.missing']])

with open('./model_interaction_full_data_fit.tex', 'w') as f:
    model_interaction_full_data_fit.to_latex(
        f, escape=False, multicolumn_format='c', multirow=True)

model_interaction_full_data_fit

Correlation Structure  GEE Independent                GEE Exchangeable  \
                              Estimate Standard Error         Estimate   
Coefficient                                                              
(Intercept), $\beta_0$        0.576457       0.117464         0.524520   
age, $\beta_1$               -0.048729       0.017763        -0.023578   
income, $\beta_2$            -0.964447       0.148575        -0.944916   
age:income, $\beta_3$         0.076834       0.022462         0.061876   

Correlation Structure                 Mixed Model                 
                       Standard Error    Estimate Standard Error  
Coefficient                                                       
(Intercept), $\beta_0$       0.115018    1.171506       0.208960  
age, $\beta_1$               0.017634   -0.054638       0.029439  
income, $\beta_2$            0.145484   -2.083890       0.269759  
age:income, $\beta_3$        0.022088    0.131501       0.036506

In [5]:
pivot_table(model_summaries[model_summaries['has.interaction'] & model_summaries['is.missing']])

Correlation Structure  GEE Independent                GEE Exchangeable  \
                              Estimate Standard Error         Estimate   
Coefficient                                                              
(Intercept), $\beta_0$        0.591668       0.159221         0.572610   
age, $\beta_1$               -0.057790       0.025625        -0.051061   
income, $\beta_2$            -0.979658       0.183379        -0.992956   
age:income, $\beta_3$         0.085894       0.029081         0.089341   

Correlation Structure                 Mixed Model                 
                       Standard Error    Estimate Standard Error  
Coefficient                                                       
(Intercept), $\beta_0$       0.145041    1.314184       0.295762  
age, $\beta_1$               0.022641   -0.115799       0.046881  
income, $\beta_2$            0.170215   -2.224775       0.344082  
age:income, $\beta_3$        0.026258    0.192534       0.051803

In [6]:
pivot_table(model_summaries[~model_summaries['has.interaction'] & ~model_summaries['is.missing']])

Correlation Structure  GEE Independent                GEE Exchangeable  \
                              Estimate Standard Error         Estimate   
Coefficient                                                              
(Intercept), $\beta_0$       -0.024537       0.070636        -0.059908   
age, $\beta_1$               -0.002281       0.010757         0.015402   

Correlation Structure                 Mixed Model                 
                       Standard Error    Estimate Standard Error  
Coefficient                                                       
(Intercept), $\beta_0$       0.068872   -0.126181       0.114182  
age, $\beta_1$               0.010418    0.028707       0.015460

In [7]:
pivot_table(model_summaries[~model_summaries['has.interaction'] & model_summaries['is.missing']])

Correlation Structure  GEE Independent                GEE Exchangeable  \
                              Estimate Standard Error         Estimate   
Coefficient                                                              
(Intercept), $\beta_0$       -0.165919       0.077793        -0.172837   
age, $\beta_1$                0.008154       0.012001         0.021671   

Correlation Structure                 Mixed Model                 
                       Standard Error    Estimate Standard Error  
Coefficient                                                       
(Intercept), $\beta_0$       0.075095   -0.362533       0.124891  
age, $\beta_1$               0.011511    0.041221       0.017344